In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

from google.colab import output
output.enable_custom_widget_manager()

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Github/sports-article-classification/

%pip install -r requirements_colab.txt
%pip install transformers
%pip install datasets
%pip install slackclient
%pip install doccano-client
%pip install wandb
%pip install scikit-multilearn
%cd src/modelling

%cd ../..
%pip install -e .


/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/Github/sports-article-classification (from -r requirements_colab.txt (line 14))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 90.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.

In [3]:
%cd Github/sports-article-classification/
%pip install -e .


[Errno 2] No such file or directory: 'Github/sports-article-classification/'
/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/Github/sports-article-classification
  Preparing metadata (setup.py) ... done
  Attempting uninstall: Sports-article-classification-packages
    Found existing installation: Sports-article-classification-packages 0.1.0
    Can't uninstall 'Sports-article-classification-packages'. No files were found to uninstall.
  Running setup.py develop for Sports-article-classification-packages


In [4]:
from src.data.preprocessing import DataPreprocessorHelland
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
#import ray.data
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from src.slack_alert.sofus_alert import sofus_alert
from huggingface_hub import login
import configparser
from src.utils import get_project_root
from src.data.dataset import get_training_data
#from ray.tune import JupyterNotebookReporter
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from transformers import EvalPrediction
import torch
from skmultilearn.model_selection import iterative_train_test_split, IterativeStratification
import wandb


from src.data.nordskog_data import get_data
from src.data.preprocessing import DataPreprocessorNordskog, DataPreprocessorHelland
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
%cd src/modelling

/content/gdrive/MyDrive/Github/sports-article-classification/src/modelling


In [6]:
data, test_experiment_data = get_data()
data_preprocessor = DataPreprocessorNordskog(data)
data_preprocessor.map_nordskog_data()
data_preprocessor.remove_extra_spaces_from_text()
data = data_preprocessor.data.copy()
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])


test_size = 0.2
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=100, shuffle=True, stratify=y)

train_split = pd.concat([X_train, y_train], axis=1)
validation_split = pd.concat([X_test, y_test], axis=1)

train_dataset = Dataset.from_pandas(train_split)
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
validation_dataset = Dataset.from_pandas(validation_split)
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])
split_dict = {'train': train_dataset, 'validation': validation_dataset}
datasets = DatasetDict(split_dict)


In [7]:
"""data = get_training_data()
data = data.loc[:, ['text', 'label']]
data_preprocessor = DataPreprocessorHelland(data['text'])
data_preprocessor.make_lower_cased()
data_preprocessor.remove_extra_spaces()
data['text'] = data_preprocessor.text_series
one_hot = MultiLabelBinarizer()
one_hot_label = one_hot.fit_transform(data['label'])
one_hot_label_list = [list(map(float, x)) for x in one_hot_label]
data['label'] = pd.Series(list(one_hot_label_list))


test_size = 0.2
X = data['text']
y = np.stack(data['label'])
stratifier = IterativeStratification(n_splits=2, order=3, sample_distribution_per_fold=[test_size, 1.0-test_size])
train_indexes, test_indexes = next(stratifier.split(X, y))

y = pd.Series(list(y), name='label')
train_split = pd.concat([X[train_indexes], y[train_indexes]], axis=1)
validation_split = pd.concat([X[test_indexes], y[test_indexes]], axis=1)

train_dataset = Dataset.from_pandas(train_split)
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
validation_dataset = Dataset.from_pandas(validation_split)
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])
split_dict = {'train': train_dataset, 'validation': validation_dataset}
datasets = DatasetDict(split_dict)"""

"data = get_training_data()\ndata = data.loc[:, ['text', 'label']]\ndata_preprocessor = DataPreprocessorHelland(data['text'])\ndata_preprocessor.make_lower_cased()\ndata_preprocessor.remove_extra_spaces()\ndata['text'] = data_preprocessor.text_series\none_hot = MultiLabelBinarizer()\none_hot_label = one_hot.fit_transform(data['label'])\none_hot_label_list = [list(map(float, x)) for x in one_hot_label]\ndata['label'] = pd.Series(list(one_hot_label_list))\n\n\ntest_size = 0.2\nX = data['text']\ny = np.stack(data['label'])\nstratifier = IterativeStratification(n_splits=2, order=3, sample_distribution_per_fold=[test_size, 1.0-test_size])\ntrain_indexes, test_indexes = next(stratifier.split(X, y))\n\ny = pd.Series(list(y), name='label')\ntrain_split = pd.concat([X[train_indexes], y[train_indexes]], axis=1)\nvalidation_split = pd.concat([X[test_indexes], y[test_indexes]], axis=1)\n\ntrain_dataset = Dataset.from_pandas(train_split)\ntrain_dataset = train_dataset.remove_columns(['__index_level

In [8]:
model_checkpoint = 'ltg/norbert2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, model_max_length=256) #256

In [9]:
def preprocess_function(examples):

    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256) #256

encoded_dataset = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/4420 [00:00<?, ? examples/s]

Map:   0%|          | 0/1106 [00:00<?, ? examples/s]

In [10]:
"""# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    #f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    score_report = classification_report(y_true,y_pred, output_dict=True, target_names=one_hot.classes_)
    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'score_report': score_report}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result"""

"# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/\ndef multi_label_metrics(predictions, labels, threshold=0.5):\n    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)\n    sigmoid = torch.nn.Sigmoid()\n    probs = sigmoid(torch.Tensor(predictions))\n    # next, use threshold to turn them into integer predictions\n    y_pred = np.zeros(probs.shape)\n    y_pred[np.where(probs >= threshold)] = 1\n    # finally, compute metrics\n    y_true = labels\n    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')\n    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')\n    #f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)\n    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')\n    accuracy = accuracy_score(y_true, y_pred)\n    score_report = classification_report(y_true,y_pred, output_dict=True, target_names=one_hot.classes_)\n    # return as dictionary\n    metr

In [11]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_class_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    #softmax = torch.nn.Softmax(dim=1)
    #probs = softmax(torch.Tensor(predictions))
    #print(softmax)
    #print(probs)
    y_pred = predictions.argmax(-1)
    print(y_pred)
    print(len(y_pred))
    # next, use threshold to turn them into integer predictions
    #y_pred = torch.round(probs)
    # finally, compute metrics
    y_true = labels

    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    #f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)
    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro', multi_class='ovo')
    accuracy = accuracy_score(y_true, y_pred)
    score_report = classification_report(y_true,y_pred, output_dict=True)
    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               #'roc_auc': roc_auc,
               'accuracy': accuracy,
               'score_report': score_report}

    return metrics

def compute_metrics(p: EvalPrediction):

    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_class_metrics(
        predictions=preds, 
        labels=p.label_ids)
    
    return result# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_class_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    #softmax = torch.nn.Softmax(dim=1)
    #probs = softmax(torch.Tensor(predictions))
    #print(softmax)
    #print(probs)
    y_pred = predictions.argmax(-1)
    print(y_pred)
    print(len(y_pred))
    # next, use threshold to turn them into integer predictions
    #y_pred = torch.round(probs)
    # finally, compute metrics
    y_true = labels

    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    #f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)
    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro', multi_class='ovo')
    accuracy = accuracy_score(y_true, y_pred)
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_class_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    #softmax = torch.nn.Softmax(dim=1)
    #probs = softmax(torch.Tensor(predictions))
    #print(softmax)
    #print(probs)
    y_pred = predictions.argmax(-1)
    print(y_pred)
    print(len(y_pred))
    # next, use threshold to turn them into integer predictions
    #y_pred = torch.round(probs)
    # finally, compute metrics
    y_true = labels

    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    #f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)
    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro', multi_class='ovo')
    accuracy = accuracy_score(y_true, y_pred)
    score_report = classification_report(y_true,y_pred, output_dict=True, target_names=label_encoder.classes_)    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               #'roc_auc': roc_auc,
               'accuracy': accuracy,
               'score_report': score_report}

    return metrics

def compute_metrics(p: EvalPrediction):

    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_class_metrics(
        predictions=preds, 
        labels=p.label_ids)
    
    return result    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               #'roc_auc': roc_auc,
               'accuracy': accuracy,
               'score_report': score_report}

    return metrics

def compute_metrics(p: EvalPrediction):

    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_class_metrics(
        predictions=preds, 
        labels=p.label_ids)
    
    return result

In [12]:
model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=12)

Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized f

In [13]:
def model_init():
    return BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=12, return_dict=True)

In [14]:
root = get_project_root()
config = configparser.ConfigParser()
config.read(root + '/src/modelling/my_config.ini')
token_hf = config['huggingface']['TOKEN']

login(token=token_hf)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50104, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [16]:
metric_name = 'accuracy'
model_name = model_checkpoint.split('/')[-1]
task = 'multi_class_classification'

args = TrainingArguments(
    output_dir = f'final-{model_name}-finetuned-{task}',
    evaluation_strategy = 'epoch',
    save_strategy = 'no',
    seed = 100,
    run_name='sports-article-classification-nordskog',
#    learning_rate=2e-5,
#    per_device_train_batch_size=batch_size,
#    per_device_eval_batch_size=batch_size,
#    num_train_epochs=5,
#    weight_decay=0.01,
#   load_best_model_at_end=False,
    #metric_for_best_model=metric_name,
    #push_to_hub=True
)

In [17]:
validation_key = "validation"
trainer = Trainer(
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized f

In [18]:
def wandb_hp_space(trial):
    return {
        'method': 'bayes',
        'metric': {'name': 'objective', 'goal': 'maximize'},
        'parameters': {
            'epoch': {'values': [3, 4, 5]},
            'learning_rate': {'distribution': 'uniform', 'min': 1e-6, 'max': 1e-4},
            'per_device_train_batch_size': {'values': [16, 32]},
            'warmup_ratio': {'distribution': 'uniform', 'min': 0, 'max': 0.5},
            'weight_decay': {'distribution': 'uniform', 'min': 0, 'max': 0.3}
        }}

In [19]:
def optimize_objective(metrics):

    #objective_metrics = ['weighted avg', '']
    objective = metrics['eval_score_report']['weighted avg']['f1-score']


    #objective_metrics = ['eval_f1_macro', 'eval_roc_auc', 'eval_accuracy']
    #objective = dict(filter(lambda item: item[0] in objective_metrics, metrics.items()))
    return objective

In [20]:
root = get_project_root()
config = configparser.ConfigParser()
config.read(root + '/src/modelling/my_config.ini')
token_wandb = config['wandb']['TOKEN']
wandb.login(key=token_wandb)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
bestrun = trainer.hyperparameter_search(
    direction='maximize', 
    backend='wandb', 
    n_trials=30, # number of trials
    hp_space=wandb_hp_space,
    compute_objective=optimize_objective,
    **{'project':'sports-article-classification-nordskog'})

Create sweep with ID: 1gzt6bqr
Sweep URL: https://wandb.ai/eirik-helland/sports-article-classification-nordskog/sweeps/1gzt6bqr


wandb: Agent Starting Run: b5u08n7q with config:
wandb: 	epoch: 5
wandb: 	learning_rate: 6.357843762126314e-05
wandb: 	per_device_train_batch_size: 16
wandb: 	warmup_ratio: 0.06933976456991225
wandb: 	weight_decay: 0.04858361921906712
wandb: Currently logged in as: eirik-helland. Use `wandb login --relogin` to force relogin


Trying to set epoch in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on an

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'Chances': {'precision': 

[11  3 11 ... 11  9  0]
1106


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'Chances': {'precision': 

[11  3 11 ... 11  9  0]
1106


eval/accuracy,▁██
eval/f1_macro,▁▇█
eval/f1_micro,▁██
eval/loss,█▁▆
eval/runtime,▅▁█
eval/samples_per_second,▄█▁
eval/steps_per_second,▄█▁
train/epoch,▁▄▅██
train/global_step,▁▄▄██
train/learning_rate,▁
train/loss,▁


In [22]:
sofus_alert()

Sofus has sent an alert - Check slack!


In [23]:
#from google.colab import runtime
#runtime.unassign()

In [24]:
from google.colab import runtime
runtime.unassign()